# Notebook to explore lat/long position of wells & find nearest neighbors
### and potentially functions to make a dataset of nearest neighbors within certain limits for each well
### and potentially functions to make a dataset of tops and other characteristics for neighboring wells.

In [21]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
welly.__version__

'0.3.0'

In [22]:
%%timeit
import os
env = %env

103 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
pd.set_option('display.max_rows', 2000)

In [24]:
picks_dic = pd.read_csv('../../SPE_006_originalData/OilSandsDB/PICKS_DIC.TXT',delimiter='\t')
picks = pd.read_csv('../../SPE_006_originalData/OilSandsDB/PICKS.TXT',delimiter='\t')
wells = pd.read_csv('../../SPE_006_originalData/OilSandsDB/WELLS.TXT',delimiter='\t')
gis = pd.read_csv('../../well_lat_lng.csv')
picks_new=picks[picks['HorID']==13000]
picks_paleoz=picks[picks['HorID']==14000]
df_new = pd.merge(wells, picks_new, on='SitID')
df_paleoz = pd.merge(wells, picks_paleoz, on='SitID')
#### NOTE: This now includes the GIS or well_lat_lng dataset too!
df_gis = pd.merge(df_paleoz, gis, on='SitID')
df_new=pd.merge(df_gis, df_new, on='SitID')
df_new.head()

,SitID,UWI (AGS)_x,UWI_x,HorID_x,Pick_x,Quality_x,UWI (AGS)_y,UWI_y,HorID_y,Pick_y,Quality_y,lat,lng,UWI (AGS),UWI,HorID,Pick,Quality
0,102496,0674010812000,00/12-08-067-01W4/0,14000,561,1,0674010812000,00/12-08-067-01W4/0,13000,475,3,54.785907,-110.129320,0674010812000,00/12-08-067-01W4/0,13000,475,3
1,102497,0674020807000,00/07-08-067-02W4/0,14000,604.5,1,0674020807000,00/07-08-067-02W4/0,13000,515,3,54.782284,-110.269446,0674020807000,00/07-08-067-02W4/0,13000,515,3
2,102498,0674021109000,00/09-11-067-02W4/0,14000,564,1,0674021109000,00/09-11-067-02W4/0,13000,480,3,54.785892,-110.186851,0674021109000,00/09-11-067-02W4/0,13000,480,3
3,102500,0674022910000,00/10-29-067-02W4/0,14000,636.5,1,0674022910000,00/10-29-067-02W4/0,13000,549,3,54.829624,-110.269422,0674022910000,00/10-29-067-02W4/0,13000,549,3
4,102501,0674023406000,00/06-34-067-02W4/0,14000,613,1,0674023406000,00/06-34-067-02W4/0,13000,529,2,54.840471,-110.224832,0674023406000,00/06-34-067-02W4/0,13000,529,2


## Possible Things to Do with Lat / Long

### Features
1. Find average pick depth of N neighbor wells within a certain maximum distance
2. Return an array of pick depths at N neighbor wells within a certain maximum distance.
3. Return the average depth above or below another known pickin N nearest neighbor wells
3. Return an array of the depths above or below another known pickin N nearest neighbor wells
4. -
5. Return the amount of range in # 2
6. -
7. Return the amount of range in # 4
8. Return if a minimum number of wells are within a certain distance
9. Return the normalized hamming distance between a window around the pick in N nearst wells and window around a point in the well in question
10. Return something about the amount of variance or max/min gr or resititivity within window in question and around pick in neighboring well(s).
11. Find pick depth of nearest well 
12. Find distance above / below known pick of pick in question in nearest well
13. Find average value of GR, resititivity, DPHI, or ? in picks of nearby well 
14. Get array of average values of some well curve within a window of pick in question in K neighor wells.
15. *Return array of UWIs of K neighbor wells.*

###  Mapping
1. Show error on map
2. Show distribution of a feature on a map.
3. Identify where jumps in pick depth occur.
4. Identify where areas of greater variance in depth occur.
5. Identify where different unsupervized groupings occur.


## Possible Steps to Chain for some of the possible features
0. Save and access pickle of all the CSVs chained into single data frame
1. Return array of UWIs of K neighbor wells, place as list in data frame
2. Place nearest well UWI of each well into new column 
3. Place nearest well pick depths into new column
4. place average pick depths of the array of wells into new column
5. do same thing for nearest well and nearest K wells but for average Gr, DPHI, and Resitivity value if present.
6. Calculate nearest and average value for distance between pick in question and known pick.

In [25]:
position = df_new[['lat','lng']]

In [26]:
position

,lat,lng
0,54.785907,-110.129320
1,54.782284,-110.269446
2,54.785892,-110.186851
3,54.829624,-110.269422
4,54.840471,-110.224832
5,54.771449,-110.402983
6,54.785901,-110.422131
7,54.800533,-110.345762
8,54.829633,-110.396621
9,54.840441,-110.320301


In [50]:
tree = neighbors.KDTree(position, leaf_size=2) 

### Saving the tree of all the lat/lng points into pickle file named 'latlng_kd_tree_size2'

In [55]:
#### dumping tree to pickle file
pickle.dump( tree, open( "latlng_kd_tree_leaf2.p", "wb" ) )


### Making sure the kd_tree worked as indended

In [35]:
dist, ind = tree.query([position][0], k=8)  

In [36]:
len(dist)

2193

In [37]:
len(ind)

2193

In [38]:
ind[4:5]

array([[ 4,  3, 64, 63,  2,  1, 65, 66]])

In [39]:
df_new[63:64]

,SitID,UWI (AGS)_x,UWI_x,HorID_x,Pick_x,Quality_x,UWI (AGS)_y,UWI_y,HorID_y,Pick_y,Quality_y,lat,lng,UWI (AGS),UWI,HorID,Pick,Quality
63,103699,0684020810000,00/10-08-068-02W4/0,14000,658.5,1,0684020810000,00/10-08-068-02W4/0,13000,572,1,54.873181,-110.269399,0684020810000,00/10-08-068-02W4/0,13000,572,1


In [40]:
df_new[4:5]

,SitID,UWI (AGS)_x,UWI_x,HorID_x,Pick_x,Quality_x,UWI (AGS)_y,UWI_y,HorID_y,Pick_y,Quality_y,lat,lng,UWI (AGS),UWI,HorID,Pick,Quality
4,102501,0674023406000,00/06-34-067-02W4/0,14000,613,1,0674023406000,00/06-34-067-02W4/0,13000,529,2,54.840471,-110.224832,0674023406000,00/06-34-067-02W4/0,13000,529,2


#### Okay, it appears KD tree has found neigbhors that are in fact near to each other

### Loading pickled tree file:

In [56]:
#### Load the previously dumped (saved) pickle file
latlng_kd_tree_leaf2 = pickle.load( open( "latlng_kd_tree_size2.p", "rb" ) )

In [57]:
#### Finding 8 nearest neighbors based on lat/long
dist, ind = latlng_kd_tree_leaf2.query([position][0], k=8)  

In [58]:
#### Testing that ind for index array and dist for array of distances between point and neighbors works after pickling
print(ind,dist)

[[   0    2   62 ...    1   65    3]
 [   1    3    4 ...    2   63   68]
 [   2    0    4 ...    3   62   63]
 ...
 [2190 2192 2181 ... 2191 2186 2180]
 [2191 2189 2192 ... 2181 2179 2188]
 [2192 2190 2188 ... 2189 2186 2184]] [[0.         0.057531   0.0874943  ... 0.14017283 0.14539497 0.14676425]
 [0.         0.04734001 0.07332214 ... 0.08267377 0.09089701 0.11681159]
 [0.         0.057531   0.06649379 ... 0.09343692 0.10808167 0.12013968]
 ...
 [0.         0.036299   0.04921231 ... 0.08492446 0.095581   0.09739753]
 [0.         0.040093   0.08332469 ... 0.11798095 0.11947642 0.13682568]
 [0.         0.036299   0.05577925 ... 0.09853038 0.10224108 0.12224832]]


## Next step - in 6 parts

A. add an object of neighbors in order of distance as new column

B. Find closest neighbor with top McMurray Pick, make new column with that pick depth

C. Find closest neighbor with base McMurray Pick, make new column with that pick depth

D. Find depth between top and base (B and C), make new column with that depth

E. Depth in D minus base McMurray in each well is nearest neighbor constant thickness prediction of top McMurray prediction

F. Do the same thing as B,C,D,E but for min, average, max of remaining neighbors of K wells

--------------------------

#### A. Add an object of neighbors in order of distance as new column

In [61]:
ind[0]

array([ 0,  2, 62, 64,  4,  1, 65,  3])

In [96]:
UWIs.head()

,UWI,Neighbors_Obj
0,00/12-08-067-01W4/0,"[{'neighbor': 1, 'distance': [0.0, 0.057531001..."
1,00/07-08-067-02W4/0,"[{'neighbor': 1, 'distance': [0.0, 0.043557006..."
2,00/09-11-067-02W4/0,"[{'neighbor': 1, 'distance': [0.0, 0.057531001..."
3,00/10-29-067-02W4/0,"[{'neighbor': 1, 'distance': [0.0, 0.043557006..."
4,00/06-34-067-02W4/0,"[{'neighbor': 1, 'distance': [0.0, 0.043557006..."


In [102]:
#### make a data frame of UWI & index from main dataframe
UWIs = df_new[['UWI']]
#UWIs.join(position, how='outer') 
UWIs_Geog = pd.concat([UWIs, position], axis=1)
UWIs_Geog.head()

,UWI,lat,lng
0,00/12-08-067-01W4/0,54.785907,-110.129320
1,00/07-08-067-02W4/0,54.782284,-110.269446
2,00/09-11-067-02W4/0,54.785892,-110.186851
3,00/10-29-067-02W4/0,54.829624,-110.269422
4,00/06-34-067-02W4/0,54.840471,-110.224832


In [128]:
#### Add object of tree for 8 neighbors to a dataframe

neighbor_array_per_well_row = []
#### for i in ind.length
for i in range(len(ind)):
    #### get the array of index values of neighbors, note: first one is the point in question which should be ignored!
    neighbors_ind = ind[i][1:]
    # for each index in that array, 
    counter = 1
    array_of_holder_obj = []
    for n in neighbors_ind:
        #### start temp object of {"neighbor":"int","UWI":"string","distance":"float"}
        holder_obj = {"neighbor":"int","UWI":"string","distance":"float"}
        #### get the distance into the temp object using index of i and each,
        holder_obj['distance'] = dist[i][counter]
        #### get the UWI using i and the dataframe
        holder_obj['UWI'] = UWIs_Geog.loc[n,'UWI']
        #### and position via "counter"
        holder_obj['neighbor'] = counter
        #### add the temp object populated as a column entry into an array
        counter = counter + 1
        array_of_holder_obj.append(holder_obj)
    #### turn array of objects into series and add as new column to dataframe
    neighbor_array_per_well_row.append(array_of_holder_obj)
UWIs_Geog['Neighbors_Obj'] = pd.Series(neighbor_array_per_well_row,index=UWIs.index)


In [115]:
#### Test results Part 1
UWIs_Geog.head()

,UWI,lat,lng,Neighbors_Obj
0,00/12-08-067-01W4/0,54.785907,-110.129320,"[{'neighbor': 1, 'distance': 0.057531001955478..."
1,00/07-08-067-02W4/0,54.782284,-110.269446,"[{'neighbor': 1, 'distance': 0.047340006083655..."
2,00/09-11-067-02W4/0,54.785892,-110.186851,"[{'neighbor': 1, 'distance': 0.057531001955478..."
3,00/10-29-067-02W4/0,54.829624,-110.269422,"[{'neighbor': 1, 'distance': 0.043557006072502..."
4,00/06-34-067-02W4/0,54.840471,-110.224832,"[{'neighbor': 1, 'distance': 0.045890364010307..."


In [127]:
#### Test results Part 1
#### Had to change display options to get this to print in full!
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000



#### Had to change display options to get this to print in full!
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print(UWIs_Geog[0:1]['Neighbors_Obj'])

height has been deprecated.

0    [{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 2, 'distance': 0.08749430324884858, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 3, 'distance': 0.10451669032744222, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 4, 'distance': 0.1099989647224123, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 5, 'distance': 0.14017282905400244, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.14539497462086076, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.14676425482044453, 'UWI': '00/10-29-067-02W4/0'}]
Name: Neighbors_Obj, dtype: object


placeholder.... title

In [129]:
df_new.head()

,SitID,UWI (AGS)_x,UWI_x,HorID_x,Pick_x,Quality_x,UWI (AGS)_y,UWI_y,HorID_y,Pick_y,Quality_y,lat,lng,UWI (AGS),UWI,HorID,Pick,Quality
0,102496,0674010812000,00/12-08-067-01W4/0,14000,561,1,0674010812000,00/12-08-067-01W4/0,13000,475,3,54.785907,-110.129320,0674010812000,00/12-08-067-01W4/0,13000,475,3
1,102497,0674020807000,00/07-08-067-02W4/0,14000,604.5,1,0674020807000,00/07-08-067-02W4/0,13000,515,3,54.782284,-110.269446,0674020807000,00/07-08-067-02W4/0,13000,515,3
2,102498,0674021109000,00/09-11-067-02W4/0,14000,564,1,0674021109000,00/09-11-067-02W4/0,13000,480,3,54.785892,-110.186851,0674021109000,00/09-11-067-02W4/0,13000,480,3
3,102500,0674022910000,00/10-29-067-02W4/0,14000,636.5,1,0674022910000,00/10-29-067-02W4/0,13000,549,3,54.829624,-110.269422,0674022910000,00/10-29-067-02W4/0,13000,549,3
4,102501,0674023406000,00/06-34-067-02W4/0,14000,613,1,0674023406000,00/06-34-067-02W4/0,13000,529,2,54.840471,-110.224832,0674023406000,00/06-34-067-02W4/0,13000,529,2


In [146]:
#### Creating a new dataframe for use below
df_new_cleaned = df_new[['SitID','HorID_x','Pick_x','Quality_x','HorID','Pick','Quality','lat','lng','UWI']].copy()

df_new_cleaned
df_new_cleaned['McMurray_Base_HorID'] = df_new_cleaned[['HorID_x']]
df_new_cleaned['McMurray_Base_HorID'] = df_new_cleaned['HorID_x']
df_new_cleaned['McMurray_Top_HorID'] = df_new_cleaned['HorID']
df_new_cleaned['McMurray_Base_DEPTH'] = df_new_cleaned['Pick_x']
df_new_cleaned['McMurray_Top_DEPTH'] = df_new_cleaned['Pick']
df_new_cleaned['McMurray_Base_Qual'] = df_new_cleaned['Quality_x']
df_new_cleaned['McMurray_Top_Qual'] = df_new_cleaned['Quality']
df_new_cleaned = df_new_cleaned[['SitID','McMurray_Base_HorID','McMurray_Top_HorID','McMurray_Base_DEPTH','McMurray_Top_DEPTH','McMurray_Base_Qual','McMurray_Top_Qual','lat','lng','UWI']]

df_new_cleaned.head()

,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,UWI
0,102496,14000,13000,561,475,1,3,54.785907,-110.129320,00/12-08-067-01W4/0
1,102497,14000,13000,604.5,515,1,3,54.782284,-110.269446,00/07-08-067-02W4/0
2,102498,14000,13000,564,480,1,3,54.785892,-110.186851,00/09-11-067-02W4/0
3,102500,14000,13000,636.5,549,1,3,54.829624,-110.269422,00/10-29-067-02W4/0
4,102501,14000,13000,613,529,1,2,54.840471,-110.224832,00/06-34-067-02W4/0


#### Merge df_new_cleaned data frame with UWIs_Geog data frame

In [185]:
df_new_cleaned_plus_nn = pd.concat([df_new_cleaned, UWIs_Geog[['Neighbors_Obj']].copy()], axis=1)
df_new_cleaned_plus_nn.head()

,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,UWI,Neighbors_Obj
0,102496,14000,13000,561,475,1,3,54.785907,-110.129320,00/12-08-067-01W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 2, 'distance': 0.08749430324884858, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 3, 'distance': 0.10451669032744222, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 4, 'distance': 0.1099989647224123, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 5, 'distance': 0.14017282905400244, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.14539497462086076, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.14676425482044453, 'UWI': '00/10-29-067-02W4/0'}]"
1,102497,14000,13000,604.5,515,1,3,54.782284,-110.269446,00/07-08-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04734000608365529, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.07332213830078355, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.07725585850924727, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 4, 'distance': 0.07846755926496476, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 5, 'distance': 0.08267376663125782, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 6, 'distance': 0.09089701215112062, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 7, 'distance': 0.11681159403073145, 'UWI': '00/06-13-068-03W4/0'}]"
2,102498,14000,13000,564,480,1,3,54.785892,-110.186851,00/09-11-067-02W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/12-08-067-01W4/0'}, {'neighbor': 2, 'distance': 0.06649378619088428, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.08267376663125782, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.08728400165551212, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 5, 'distance': 0.09343691917544036, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 6, 'distance': 0.10808166919974113, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 7, 'distance': 0.12013967631468725, 'UWI': '00/10-08-068-02W4/0'}]"
3,102500,14000,13000,636.5,549,1,3,54.829624,-110.269422,00/10-29-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04355700607250212, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 2, 'distance': 0.04589036401030715, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.04734000608365529, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.052016152587450945, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 5, 'distance': 0.07898840474399109, 'UWI': '00/06-13-068-03W4/0'}, {'neighbor': 6, 'distance': 0.08169505420160347, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 7, 'distance': 0.08752301126560716, 'UWI': '00/11-29-068-02W4/0'}]"
4,102501,14000,13000,613,529,1,2,54.840471,-110.224832,00/06-34-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04589036401030715, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.050134469469623355, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 3, 'distance': 0.055282561346227096, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 4, 'distance': 0.06649378619088428, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 5, 'distance': 0.07332213830078355, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.07715524006184091, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.09180352126144718, 'UWI': '00/11-29-068-02W4/0'}]"


#### This is how to access the data on nearest neigbhors in the 'Neighbors_Obj' column

In [228]:
#### The object for the 1st neighbor of the 1 index well
df_new_cleaned_plus_nn.loc[1:1,'Neighbors_Obj'][1][1]

{'UWI': '00/06-34-067-02W4/0', 'distance': 0.07332213830078355, 'neighbor': 2}

In [217]:
#### The distance from the well in question for that row to the 1st neighbor 
df_new_cleaned_plus_nn.loc[1:1,'Neighbors_Obj'][1][3]['UWI']

'00/10-14-067-03W4/0'

In [235]:
df_new_cleaned_plus_nn.iloc[0,10][1]['UWI']

'00/11-08-068-01W4/0'

In [237]:
### col 3 is McMurray_Base_DEPTH
df_new_cleaned_plus_nn.iloc[0,3]

'561'

In [238]:
### col 4 is McMurray_Top_DEPTH
df_new_cleaned_plus_nn.iloc[0,4]

'475'

In [251]:
test = df_new_cleaned_plus_nn.set_index('UWI').index.get_loc('00/06-34-067-02W4/0')
print(type(test),test)

<class 'int'> 4


,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,UWI,Neighbors_Obj
0,102496,14000,13000,561,475,1,3,54.785907,-110.129320,00/12-08-067-01W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 2, 'distance': 0.08749430324884858, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 3, 'distance': 0.10451669032744222, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 4, 'distance': 0.1099989647224123, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 5, 'distance': 0.14017282905400244, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.14539497462086076, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.14676425482044453, 'UWI': '00/10-29-067-02W4/0'}]"
1,102497,14000,13000,604.5,515,1,3,54.782284,-110.269446,00/07-08-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04734000608365529, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.07332213830078355, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.07725585850924727, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 4, 'distance': 0.07846755926496476, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 5, 'distance': 0.08267376663125782, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 6, 'distance': 0.09089701215112062, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 7, 'distance': 0.11681159403073145, 'UWI': '00/06-13-068-03W4/0'}]"
2,102498,14000,13000,564,480,1,3,54.785892,-110.186851,00/09-11-067-02W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/12-08-067-01W4/0'}, {'neighbor': 2, 'distance': 0.06649378619088428, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.08267376663125782, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.08728400165551212, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 5, 'distance': 0.09343691917544036, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 6, 'distance': 0.10808166919974113, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 7, 'distance': 0.12013967631468725, 'UWI': '00/10-08-068-02W4/0'}]"
3,102500,14000,13000,636.5,549,1,3,54.829624,-110.269422,00/10-29-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04355700607250212, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 2, 'distance': 0.04589036401030715, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.04734000608365529, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.052016152587450945, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 5, 'distance': 0.07898840474399109, 'UWI': '00/06-13-068-03W4/0'}, {'neighbor': 6, 'distance': 0.08169505420160347, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 7, 'distance': 0.08752301126560716, 'UWI': '00/11-29-068-02W4/0'}]"
4,102501,14000,13000,613,529,1,2,54.840471,-110.224832,00/06-34-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04589036401030715, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.050134469469623355, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 3, 'distance': 0.055282561346227096, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 4, 'distance': 0.06649378619088428, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 5, 'distance': 0.07332213830078355, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.07715524006184091, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.09180352126144718, 'UWI': '00/11-29-068-02W4/0'}]"
5,102503,14000,13000,553.5,488.5,1,2,54.771449,-110.402983,00/11-04-067-03W4/0,"[{'neighbor': 1, 'distance': 0.023989710460949984, 'UWI': '00/10-08-067-03W4/0'}, {'neighbor': 2, 'distance': 0.058530785916471276, 'UWI': '00/10-28-067-03W4/0'}, {'neighbor': 3, 'distance': 0.06418817567901171, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 4, 'distance': 0.08773076140100079, 'UWI': '00/11-24-067-04W4/0'}, {'neighbor': 5, 'distance': 0.08820637669692524, 'UWI': '00/09-05-068-03W4/0'}, {'neighbor': 6, 'distance': 0.09543100002094827, 'UWI': '00/10-02-067-04W4/0'}, {'neighbor': 7, 'distance': 0.10768569630179346, 'UWI': '00/07-36-067-03W4/0'}]"
6,102505,140

In [223]:
df_new_cleaned_plus_nn.loc[1:2,'Neighbors_Obj'][2]

[{'UWI': '00/12-08-067-01W4/0',
  'distance': 0.05753100195547899,
  'neighbor': 1},
 {'UWI': '00/06-34-067-02W4/0',
  'distance': 0.06649378619088428,
  'neighbor': 2},
 {'UWI': '00/07-08-067-02W4/0',
  'distance': 0.08267376663125782,
  'neighbor': 3},
 {'UWI': '00/09-11-068-02W4/0',
  'distance': 0.08728400165551212,
  'neighbor': 4},
 {'UWI': '00/10-29-067-02W4/0',
  'distance': 0.09343691917544036,
  'neighbor': 5},
 {'UWI': '00/11-08-068-01W4/0',
  'distance': 0.10808166919974113,
  'neighbor': 6},
 {'UWI': '00/10-08-068-02W4/0',
  'distance': 0.12013967631468725,
  'neighbor': 7}]

### B. Find closest neighbor with top McMurray Pick, make new column with that pick depth

In [194]:
index_of_neigh = df_new_cleaned_plus_nn.set_index('UWI').index.get_loc('00/09-11-067-02W4/0')
index_of_neigh

2

In [246]:
df_new_cleaned_plus_nn.iloc[0,10][1]['UWI']

'00/11-08-068-01W4/0'

In [273]:
df_new_cleaned_plus_nn['SitID'][1]

102497

In [292]:
#### Neighbors_Obj must be 10th Col!!

def broadcastFuncForFindNearestNPickDepth(df,pickColInt,newPickColName):
    #### For each row in dataframe,
    df[newPickColName] = np.nan
    print( df.iloc[0:1][newPickColName])
    for eachRow in range(len(df)):
        #### Find the nearest neighbor UWI
        print(eachRow)
        UWI = df.iloc[eachRow,10][1]['UWI']
        print(type(UWI),"type to right of ",UWI)
        #### With the UWI from above, find the pickDepthName
        index_of_neigh_UWI = df.set_index('UWI').index.get_loc(UWI)
        print("index_of_neigh_UWI= ",index_of_neigh_UWI)
        pick_depth = df.iloc[index_of_neigh_UWI,pickColInt]
        print("pick_depth ",pick_depth)
        try:
            pick_depth = float(pick_depth)
        except:
            UWI = df.iloc[eachRow,10][2]['UWI']
            index_of_neigh_UWI = df.set_index('UWI').index.get_loc(UWI)
            pick_depth = df.iloc[index_of_neigh_UWI,pickColInt]
            try:
                pick_depth = float(pick_depth)
            except:
                UWI = df.iloc[eachRow,10][3]['UWI']
                index_of_neigh_UWI = df.set_index('UWI').index.get_loc(UWI)
                pick_depth = df.iloc[index_of_neigh_UWI,pickColInt]
                try:
                    pick_depth = float(pick_depth)
                except:
                    UWI = df.iloc[eachRow,10][4]['UWI']
                    index_of_neigh_UWI = df.set_index('UWI').index.get_loc(UWI)
                    pick_depth = df.iloc[index_of_neigh_UWI,pickColInt]
                    try:
                        pick_depth = float(pick_depth)
                    except:
                        pick_depth = "no_pick"
            
        #### Write the pickDepthName to a new column
#         df[newPickColName][eachRow] = pick_depth
        eachRowP1 = eachRow+1
        df.iloc[eachRow:eachRowP1][newPickColName] = pick_depth
    return df

In [293]:
temp2 = broadcastFuncForFindNearestNPickDepth(df_new_cleaned_plus_nn,4,'NN1_McMurray_Top_DEPTH')

0   NaN
Name: NN1_McMurray_Top_DEPTH, dtype: float64
0
<class 'str'> type to right of  00/11-08-068-01W4/0
index_of_neigh_UWI=  62
pick_depth  544
1
<class 'str'> type to right of  00/06-34-067-02W4/0
index_of_neigh_UWI=  4
pick_depth  529


/Users/justingosses/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2
<class 'str'> type to right of  00/06-34-067-02W4/0
index_of_neigh_UWI=  4
pick_depth  529
3
<class 'str'> type to right of  00/06-34-067-02W4/0
index_of_neigh_UWI=  4
pick_depth  529
4
<class 'str'> type to right of  00/09-11-068-02W4/0
index_of_neigh_UWI=  64
pick_depth  514
5
<class 'str'> type to right of  00/10-28-067-03W4/0
index_of_neigh_UWI=  8
pick_depth  493.5
6
<class 'str'> type to right of  00/10-28-067-03W4/0
index_of_neigh_UWI=  8
pick_depth  493.5
7
<class 'str'> type to right of  00/10-28-067-03W4/0
index_of_neigh_UWI=  8
pick_depth  493.5
8
<class 'str'> type to right of  00/10-08-067-03W4/0
index_of_neigh_UWI=  6
pick_depth  501.5
9
<class 'str'> type to right of  00/10-14-067-03W4/0
index_of_neigh_UWI=  7
pick_depth  553.5
10
<class 'str'> type to right of  00/10-08-067-03W4/0
index_of_neigh_UWI=  6
pick_depth  501.5
11
<class 'str'> type to right of  00/07-33-067-04W4/0
index_of_neigh_UWI=  13
pick_depth  509
12
<class 'str'> type to right of  00/11-02-068-04W4/0

88
<class 'str'> type to right of  00/15-05-068-08W4/0
index_of_neigh_UWI=  87
pick_depth  511
89
<class 'str'> type to right of  00/08-11-069-08W4/0
index_of_neigh_UWI=  140
pick_depth  442
90
<class 'str'> type to right of  00/16-21-068-08W4/0
index_of_neigh_UWI=  88
pick_depth  481
91
<class 'str'> type to right of  00/07-06-069-09W4/0
index_of_neigh_UWI=  143
pick_depth  630
92
<class 'str'> type to right of  00/07-06-069-09W4/0
index_of_neigh_UWI=  143
pick_depth  630
93
<class 'str'> type to right of  00/06-27-067-10W4/0
index_of_neigh_UWI=  30
pick_depth  518
94
<class 'str'> type to right of  00/10-12-069-10W4/0
index_of_neigh_UWI=  147
pick_depth  641.3
95
<class 'str'> type to right of  00/13-09-068-11W4/0
index_of_neigh_UWI=  96
pick_depth  455
96
<class 'str'> type to right of  00/10-33-067-11W4/0
index_of_neigh_UWI=  33
pick_depth  509
97
<class 'str'> type to right of  00/13-09-068-11W4/0
index_of_neigh_UWI=  96
pick_depth  455
98
<class 'str'> type to right of  00/13-09-

173
<class 'str'> type to right of  00/07-24-069-17W4/0
index_of_neigh_UWI=  176
pick_depth          
174
<class 'str'> type to right of  00/11-06-070-15W4/0
index_of_neigh_UWI=  214
pick_depth  455
175
<class 'str'> type to right of  00/06-30-069-17W4/0
index_of_neigh_UWI=  177
pick_depth  472.5
176
<class 'str'> type to right of  00/11-32-068-16W4/0
index_of_neigh_UWI=  115
pick_depth          
177
<class 'str'> type to right of  00/11-21-069-17W4/0
index_of_neigh_UWI=  175
pick_depth  475
178
<class 'str'> type to right of  00/06-30-069-17W4/0
index_of_neigh_UWI=  177
pick_depth  472.5
179
<class 'str'> type to right of  00/06-30-069-17W4/0
index_of_neigh_UWI=  177
pick_depth  472.5
180
<class 'str'> type to right of  00/06-32-069-04W4/0
index_of_neigh_UWI=  129
pick_depth  476.5
181
<class 'str'> type to right of  00/12-18-071-04W4/0
index_of_neigh_UWI=  280
pick_depth  450
182
<class 'str'> type to right of  00/05-21-071-04W4/0
index_of_neigh_UWI=  281
pick_depth  459
183
<class '

256
<class 'str'> type to right of  00/11-16-071-26W4/0
index_of_neigh_UWI=  368
pick_depth  619
257
<class 'str'> type to right of  00/06-18-070-26W4/0
index_of_neigh_UWI=  254
pick_depth  665.07
258
<class 'str'> type to right of  00/06-18-070-26W4/0
index_of_neigh_UWI=  254
pick_depth  665.07
259
<class 'str'> type to right of  00/11-32-070-01W5/0
index_of_neigh_UWI=  262
pick_depth  622.5
260
<class 'str'> type to right of  00/11-12-070-27W4/0
index_of_neigh_UWI=  258
pick_depth  679.7
261
<class 'str'> type to right of  00/06-14-070-01W5/0
index_of_neigh_UWI=  260
pick_depth  646.18
262
<class 'str'> type to right of  00/05-07-071-01W5/0
index_of_neigh_UWI=  371
pick_depth  612
263
<class 'str'> type to right of  00/11-07-070-01W5/0
index_of_neigh_UWI=  259
pick_depth  612
264
<class 'str'> type to right of  00/11-32-070-02W5/0
index_of_neigh_UWI=  265
pick_depth          
265
<class 'str'> type to right of  00/11-12-071-03W5/2
index_of_neigh_UWI=  378
pick_depth          
266
<cl

index_of_neigh_UWI=  344
pick_depth  486
341
<class 'str'> type to right of  02/10-12-072-19W4/0
index_of_neigh_UWI=  453
pick_depth  491.34
342
<class 'str'> type to right of  00/11-34-071-20W4/0
index_of_neigh_UWI=  345
pick_depth  501.4
343
<class 'str'> type to right of  00/11-34-071-20W4/0
index_of_neigh_UWI=  345
pick_depth  501.4
344
<class 'str'> type to right of  00/10-15-071-20W4/0
index_of_neigh_UWI=  342
pick_depth  484.5
345
<class 'str'> type to right of  00/06-02-072-20W4/0
index_of_neigh_UWI=  456
pick_depth  492.25
346
<class 'str'> type to right of  00/06-11-071-22W4/0
index_of_neigh_UWI=  351
pick_depth  629.72
347
<class 'str'> type to right of  02/07-26-071-21W4/0
index_of_neigh_UWI=  348
pick_depth          
348
<class 'str'> type to right of  00/11-22-071-21W4/0
index_of_neigh_UWI=  347
pick_depth          
349
<class 'str'> type to right of  00/11-22-071-21W4/0
index_of_neigh_UWI=  347
pick_depth          
350
<class 'str'> type to right of  00/07-24-071-23W4/0


KeyboardInterrupt: 

In [285]:
temp2.describe()

,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,"(0, NN1_McMurray_Top_DEPTH)","(1, NN1_McMurray_Top_DEPTH)","(2, NN1_McMurray_Top_DEPTH)","(3, NN1_McMurray_Top_DEPTH)","(4, NN1_McMurray_Top_DEPTH)","(5, NN1_McMurray_Top_DEPTH)","(6, NN1_McMurray_Top_DEPTH)","(7, NN1_McMurray_Top_DEPTH)","(8, NN1_McMurray_Top_DEPTH)","(9, NN1_McMurray_Top_DEPTH)","(10, NN1_McMurray_Top_DEPTH)","(11, NN1_McMurray_Top_DEPTH)","(12, NN1_McMurray_Top_DEPTH)","(13, NN1_McMurray_Top_DEPTH)","(14, NN1_McMurray_Top_DEPTH)","(15, NN1_McMurray_Top_DEPTH)","(16, NN1_McMurray_Top_DEPTH)","(17, NN1_McMurray_Top_DEPTH)","(18, NN1_McMurray_Top_DEPTH)","(19, NN1_McMurray_Top_DEPTH)","(20, NN1_McMurray_Top_DEPTH)","(21, NN1_McMurray_Top_DEPTH)","(22, NN1_McMurray_Top_DEPTH)","(23, NN1_McMurray_Top_DEPTH)","(24, NN1_McMurray_Top_DEPTH)","(25, NN1_McMurray_Top_DEPTH)","(26, NN1_McMurray_Top_DEPTH)","(27, NN1_McMurray_Top_DEPTH)","(28, NN1_McMurray_Top_DEPTH)","(29, NN1_McMurray_Top_DEPTH)","(30, NN1_McMurray_Top_DEPTH)","(31, NN1_McMurray_Top_DEPTH)","(32, NN1_McMurray_Top_DEPTH)","(33, NN1_McMurray_Top_DEPTH)","(34, NN1_McMurray_Top_DEPTH)","(35, NN1_McMurray_Top_DEPTH)","(36, NN1_McMurray_Top_DEPTH)","(37, NN1_McMurray_Top_DEPTH)","(38, NN1_McMurray_Top_DEPTH)","(39, NN1_McMurray_Top_DEPTH)","(40, NN1_McMurray_Top_DEPTH)","(41, NN1_McMurray_Top_DEPTH)","(42, NN1_McMurray_Top_DEPTH)","(43, NN1_McMurray_Top_DEPTH)","(44, NN1_McMurray_Top_DEPTH)","(45, NN1_McMurray_Top_DEPTH)","(46, NN1_McMurray_Top_DEPTH)","(47, NN1_McMurray_Top_DEPTH)","(48, NN1_McMurray_Top_DEPTH)","(49, NN1_McMurray_Top_DEPTH)","(50, NN1_McMurray_Top_DEPTH)","(51, NN1_McMurray_Top_DEPTH)","(52, NN1_McMurray_Top_DEPTH)","(53, NN1_McMurray_Top_DEPTH)","(54, NN1_McMurray_Top_DEPTH)","(56, NN1_McMurray_Top_DEPTH)","(57, NN1_McMurray_Top_DEPTH)","(58, NN1_McMurray_Top_DEPTH)","(59, NN1_McMurray_Top_DEPTH)","(60, NN1_McMurray_Top_DEPTH)","(61, NN1_McMurray_Top_DEPTH)","(62, NN1_McMurray_Top_DEPTH)","(63, NN1_McMurray_Top_DEPTH)","(64, NN1_McMurray_Top_DEPTH)","(65, NN1_McMurray_Top_DEPTH)","(66, NN1_McMurray_Top_DEPTH)","(67, NN1_McMurray_Top_DEPTH)","(68, NN1_McMurray_Top_DEPTH)","(69, NN1_McMurray_Top_DEPTH)","(70, NN1_McMurray_Top_DEPTH)","(71, NN1_McMurray_Top_DEPTH)","(72, NN1_McMurray_Top_DEPTH)","(73, NN1_McMurray_Top_DEPTH)","(74, NN1_McMurray_Top_DEPTH)","(75, NN1_McMurray_Top_DEPTH)","(76, NN1_McMurray_Top_DEPTH)","(77, NN1_McMurray_Top_DEPTH)","(78, NN1_McMurray_Top_DEPTH)","(79, NN1_McMurray_Top_DEPTH)","(80, NN1_McMurray_Top_DEPTH)","(81, NN1_McMurray_Top_DEPTH)","(82, NN1_McMurray_Top_DEPTH)","(83, NN1_McMurray_Top_DEPTH)","(84, NN1_McMurray_Top_DEPTH)","(85, NN1_McMurray_Top_DEPTH)","(86, NN1_McMurray_Top_DEPTH)","(87, NN1_McMurray_Top_DEPTH)","(88, NN1_McMurray_Top_DEPTH)","(89, NN1_McMurray_Top_DEPTH)","(90, NN1_McMurray_Top_DEPTH)","(91, NN1_McMurray_Top_DEPTH)","(92, NN1_McMurray_Top_DEPTH)","(93, NN1_McMurray_Top_DEPTH)","(94, NN1_McMurray_Top_DEPTH)","(95, NN1_McMurray_Top_DEPTH)","(96, NN1_McMurray_Top_DEPTH)","(97, NN1_McMurray_Top_DEPTH)","(98, NN1_McMurray_Top_DEPTH)","(99, NN1_McMurray_Top_DEPTH)","(100, NN1_McMurray_Top_DEPTH)","(101, NN1_McMurray_Top_DEPTH)","(102, NN1_McMurray_Top_DEPTH)","(103, NN1_McMurray_Top_DEPTH)","(104, NN1_McMurray_Top_DEPTH)","(105, NN1_McMurray_Top_DEPTH)","(106, NN1_McMurray_Top_DEPTH)","(107, NN1_McMurray_Top_DEPTH)","(108, NN1_McMurray_Top_DEPTH)","(109, NN1_McMurray_Top_DEPTH)","(110, NN1_McMurray_Top_DEPTH)","(111, NN1_McMurray_Top_DEPTH)","(112, NN1_McMurray_Top_DEPTH)","(113, NN1_McMurray_Top_DEPTH)","(114, NN1_McMurray_Top_DEPTH)","(115, NN1_McMurray_Top_DEPTH)","(116, NN1_McMurray_Top_DEPTH)","(117, NN1_McMurray_Top_DEPTH)","(118, NN1_McMurray_Top_DEPTH)","(119, NN1_McMurray_Top_DEPTH)","(120, NN1_McMurray_Top_DEPTH)","(121, NN1_McMurray_Top_DEPTH)","(122, NN1_McMurray_Top_DEPTH)","(123, NN1_McMurray_Top_DEPTH)","(124, NN1_McMurray_Top_DEPTH)","(125, NN1_McMurray_Top_DEPTH)","(126, NN1_McMurra

In [290]:
temp3 = temp2[['SitID','McMurray_Base_HorID','McMurray_Top_HorID','McMurray_Base_DEPTH','McMurray_Top_DEPTH','McMurray_Base_Qual','McMurray_Top_Qual','lat','lng','UWI','Neighbors_Obj','NN1_McMurray_Top_DEPTH']]

In [291]:
temp3.head()

,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,UWI,Neighbors_Obj,NN1_McMurray_Top_DEPTH
0,102496,14000,13000,561,475,1,3,54.785907,-110.129320,00/12-08-067-01W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 2, 'distance': 0.08749430324884858, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 3, 'distance': 0.10451669032744222, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 4, 'distance': 0.1099989647224123, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 5, 'distance': 0.14017282905400244, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.14539497462086076, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.14676425482044453, 'UWI': '00/10-29-067-02W4/0'}]",544.00
1,102497,14000,13000,604.5,515,1,3,54.782284,-110.269446,00/07-08-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04734000608365529, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.07332213830078355, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.07725585850924727, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 4, 'distance': 0.07846755926496476, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 5, 'distance': 0.08267376663125782, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 6, 'distance': 0.09089701215112062, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 7, 'distance': 0.11681159403073145, 'UWI': '00/06-13-068-03W4/0'}]",529.00
2,102498,14000,13000,564,480,1,3,54.785892,-110.186851,00/09-11-067-02W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/12-08-067-01W4/0'}, {'neighbor': 2, 'distance': 0.06649378619088428, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.08267376663125782, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.08728400165551212, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 5, 'distance': 0.09343691917544036, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 6, 'distance': 0.10808166919974113, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 7, 'distance': 0.12013967631468725, 'UWI': '00/10-08-068-02W4/0'}]",529.00
3,102500,14000,13000,636.5,549,1,3,54.829624,-110.269422,00/10-29-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04355700607250212, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 2, 'distance': 0.04589036401030715, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.04734000608365529, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.052016152587450945, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 5, 'distance': 0.07898840474399109, 'UWI': '00/06-13-068-03W4/0'}, {'neighbor': 6, 'distance': 0.08169505420160347, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 7, 'distance': 0.08752301126560716, 'UWI': '00/11-29-068-02W4/0'}]",529.00
4,102501,14000,13000,613,529,1,2,54.840471,-110.224832,00/06-34-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04589036401030715, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.050134469469623355, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 3, 'distance': 0.055282561346227096, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 4, 'distance': 0.06649378619088428, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 5, 'distance': 0.07332213830078355, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.07715524006184091, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.09180352126144718, 'UWI': '00/11-29-068-02W4/0'}]",514.00
5,102503,14000,13000,553.5,488.5,1,2,54.771449,-110.402983,00/11-04-067-03W4/0,"[{'neighbor': 1, 'distance': 0.023989710460949984, 'UWI': '00/10-08-067-03W4/0'}, {'neighbor': 2, 'distance': 0.058530785916471276, 'UWI': '00/10-28-067-03W4/0'}, {'neighbor': 3, 'distance': 0.06418817567901171, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 4, 'distance': 0.08773076140100079, 'UWI': '00/11-24-067-04W4/0'}, {'neighbor': 5, 'distance': 0.08820637669692524, 'UWI': '00/09-05-068-03W4/0'}, {'neighbor': 6, 'distance': 0.09543100002094827, 'UWI': '00/10-02-067-04W4/0'}, {'neighbor': 7, 'distance': 0.10768

In [294]:
temp4 = broadcastFuncForFindNearestNPickDepth(temp3,3,'NN1_McMurray_Base_DEPTH')

/Users/justingosses/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/justingosses/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0   NaN
Name: NN1_McMurray_Base_DEPTH, dtype: float64
0
<class 'str'> type to right of  00/11-08-068-01W4/0
index_of_neigh_UWI=  62
pick_depth  630
1
<class 'str'> type to right of  00/06-34-067-02W4/0
index_of_neigh_UWI=  4
pick_depth  613
2
<class 'str'> type to right of  00/06-34-067-02W4/0
index_of_neigh_UWI=  4
pick_depth  613
3
<class 'str'> type to right of  00/06-34-067-02W4/0
index_of_neigh_UWI=  4
pick_depth  613
4
<class 'str'> type to right of  00/09-11-068-02W4/0
index_of_neigh_UWI=  64
pick_depth  603
5
<class 'str'> type to right of  00/10-28-067-03W4/0
index_of_neigh_UWI=  8
pick_depth  558
6
<class 'str'> type to right of  00/10-28-067-03W4/0
index_of_neigh_UWI=  8
pick_depth  558
7
<class 'str'> type to right of  00/10-28-067-03W4/0
index_of_neigh_UWI=  8
pick_depth  558
8
<class 'str'> type to right of  00/10-08-067-03W4/0
index_of_neigh_UWI=  6
pick_depth  572.5
9
<class 'str'> type to right of  00/10-14-067-03W4/0
index_of_neigh_UWI=  7
pick_depth  606.5
10
<class 

86
<class 'str'> type to right of  00/14-05-068-07W4/0
index_of_neigh_UWI=  83
pick_depth  508
87
<class 'str'> type to right of  00/16-21-068-08W4/0
index_of_neigh_UWI=  88
pick_depth  540
88
<class 'str'> type to right of  00/15-05-068-08W4/0
index_of_neigh_UWI=  87
pick_depth  566.5
89
<class 'str'> type to right of  00/08-11-069-08W4/0
index_of_neigh_UWI=  140
pick_depth  501.5
90
<class 'str'> type to right of  00/16-21-068-08W4/0
index_of_neigh_UWI=  88
pick_depth  540
91
<class 'str'> type to right of  00/07-06-069-09W4/0
index_of_neigh_UWI=  143
pick_depth  681.5
92
<class 'str'> type to right of  00/07-06-069-09W4/0
index_of_neigh_UWI=  143
pick_depth  681.5
93
<class 'str'> type to right of  00/06-27-067-10W4/0
index_of_neigh_UWI=  30
pick_depth  567.5
94
<class 'str'> type to right of  00/10-12-069-10W4/0
index_of_neigh_UWI=  147
pick_depth  690.07
95
<class 'str'> type to right of  00/13-09-068-11W4/0
index_of_neigh_UWI=  96
pick_depth  481
96
<class 'str'> type to right of

171
<class 'str'> type to right of  00/07-24-069-16W4/0
index_of_neigh_UWI=  174
pick_depth  463.3
172
<class 'str'> type to right of  00/11-32-068-16W4/0
index_of_neigh_UWI=  115
pick_depth  449.88
173
<class 'str'> type to right of  00/07-24-069-17W4/0
index_of_neigh_UWI=  176
pick_depth  440
174
<class 'str'> type to right of  00/11-06-070-15W4/0
index_of_neigh_UWI=  214
pick_depth  491
175
<class 'str'> type to right of  00/06-30-069-17W4/0
index_of_neigh_UWI=  177
pick_depth  478
176
<class 'str'> type to right of  00/11-32-068-16W4/0
index_of_neigh_UWI=  115
pick_depth  449.88
177
<class 'str'> type to right of  00/11-21-069-17W4/0
index_of_neigh_UWI=  175
pick_depth  487
178
<class 'str'> type to right of  00/06-30-069-17W4/0
index_of_neigh_UWI=  177
pick_depth  478
179
<class 'str'> type to right of  00/06-30-069-17W4/0
index_of_neigh_UWI=  177
pick_depth  478
180
<class 'str'> type to right of  00/06-32-069-04W4/0
index_of_neigh_UWI=  129
pick_depth  537
181
<class 'str'> type

256
<class 'str'> type to right of  00/11-16-071-26W4/0
index_of_neigh_UWI=  368
pick_depth  659
257
<class 'str'> type to right of  00/06-18-070-26W4/0
index_of_neigh_UWI=  254
pick_depth  706.83
258
<class 'str'> type to right of  00/06-18-070-26W4/0
index_of_neigh_UWI=  254
pick_depth  706.83
259
<class 'str'> type to right of  00/11-32-070-01W5/0
index_of_neigh_UWI=  262
pick_depth  650.5
260
<class 'str'> type to right of  00/11-12-070-27W4/0
index_of_neigh_UWI=  258
pick_depth  717.5
261
<class 'str'> type to right of  00/06-14-070-01W5/0
index_of_neigh_UWI=  260
pick_depth  673
262
<class 'str'> type to right of  00/05-07-071-01W5/0
index_of_neigh_UWI=  371
pick_depth  624
263
<class 'str'> type to right of  00/11-07-070-01W5/0
index_of_neigh_UWI=  259
pick_depth  628
264
<class 'str'> type to right of  00/11-32-070-02W5/0
index_of_neigh_UWI=  265
pick_depth  631.85
265
<class 'str'> type to right of  00/11-12-071-03W5/2
index_of_neigh_UWI=  378
pick_depth  649.22
266
<class 'st

...

1842
<class 'str'> type to right of  00/10-14-091-20W4/0
index_of_neigh_UWI=  1844
pick_depth  229.21
1843
<class 'str'> type to right of  00/11-13-091-21W4/0
index_of_neigh_UWI=  1847
pick_depth  225
1844
<class 'str'> type to right of  00/14-34-090-20W4/0
index_of_neigh_UWI=  1800
pick_depth  231
1845
<class 'str'> type to right of  00/11-03-091-20W4/0
index_of_neigh_UWI=  1842
pick_depth  225
1846
<class 'str'> type to right of  00/11-07-091-20W4/0
index_of_neigh_UWI=  1843
pick_depth  230
1847
<class 'str'> type to right of  00/11-07-091-20W4/0
index_of_neigh_UWI=  1843
pick_depth  230
1848
<class 'str'> type to right of  AA/16-21-092-08W4/0
index_of_neigh_UWI=  1850
pick_depth  164.9
1849
<class 'str'> type to right of  AA/10-36-091-09W4/0
index_of_neigh_UWI=  1815
pick_depth  128.93
1850
<class 'str'> type to right of  AA/13-02-092-08W4/0
index_of_neigh_UWI=  1848
pick_depth  192.02
1851
<class 'str'> type to right of  AA/13-02-092-08W4/0
index_of_neigh_UWI=  1848
pick_depth  192

1925
<class 'str'> type to right of  AA/10-29-092-17W4/0
index_of_neigh_UWI=  1876
pick_depth  253.59
1926
<class 'str'> type to right of  00/07-36-093-19W4/0
index_of_neigh_UWI=  1932
pick_depth  240
1927
<class 'str'> type to right of  00/10-25-092-18W4/0
index_of_neigh_UWI=  1879
pick_depth  264.5
1928
<class 'str'> type to right of  00/07-36-093-19W4/0
index_of_neigh_UWI=  1932
pick_depth  240
1929
<class 'str'> type to right of  00/07-22-093-19W4/0
index_of_neigh_UWI=  1930
pick_depth  241.71
1930
<class 'str'> type to right of  00/14-03-093-19W4/0
index_of_neigh_UWI=  1929
pick_depth  258
1931
<class 'str'> type to right of  00/07-36-093-19W4/0
index_of_neigh_UWI=  1932
pick_depth  240
1932
<class 'str'> type to right of  00/11-31-093-18W4/0
index_of_neigh_UWI=  1928
pick_depth  242
1933
<class 'str'> type to right of  00/05-36-092-20W4/0
index_of_neigh_UWI=  1888
pick_depth  252
1934
<class 'str'> type to right of  00/05-25-093-20W4/0
index_of_neigh_UWI=  1935
pick_depth  248.11

2010
<class 'str'> type to right of  AB/06-17-094-09W4/0
index_of_neigh_UWI=  1954
pick_depth  60
2011
<class 'str'> type to right of  AB/03-12-096-10W4/0
index_of_neigh_UWI=  2062
pick_depth  76.5
2012
<class 'str'> type to right of  AB/16-28-095-10W4/0
index_of_neigh_UWI=  2014
pick_depth  75
2013
<class 'str'> type to right of  AB/16-28-095-10W4/0
index_of_neigh_UWI=  2014
pick_depth  75
2014
<class 'str'> type to right of  AA/04-15-095-10W4/0
index_of_neigh_UWI=  2013
pick_depth  78.33
2015
<class 'str'> type to right of  AB/10-18-096-10W4/0
index_of_neigh_UWI=  2063
pick_depth  113
2016
<class 'str'> type to right of  AA/03-30-095-11W4/0
index_of_neigh_UWI=  2018
pick_depth  62.18
2017
<class 'str'> type to right of  AA/11-18-095-11W4/0
index_of_neigh_UWI=  2016
pick_depth  64.92
2018
<class 'str'> type to right of  AA/11-18-095-11W4/0
index_of_neigh_UWI=  2016
pick_depth  64.92
2019
<class 'str'> type to right of  AA/08-31-095-10W4/0
index_of_neigh_UWI=  2015
pick_depth  78
2020


2094
<class 'str'> type to right of  00/07-06-095-21W4/0
index_of_neigh_UWI=  2043
pick_depth  504
2095
<class 'str'> type to right of  AA/10-33-097-06W4/0
index_of_neigh_UWI=  2098
pick_depth  59
2096
<class 'str'> type to right of  AA/10-33-097-06W4/0
index_of_neigh_UWI=  2098
pick_depth  59
2097
<class 'str'> type to right of  AA/11-06-098-06W4/0
index_of_neigh_UWI=  2130
pick_depth  35
2098
<class 'str'> type to right of  AA/10-29-097-06W4/0
index_of_neigh_UWI=  2097
pick_depth  89.5
2099
<class 'str'> type to right of  AA/05-26-096-07W4/0
index_of_neigh_UWI=  2050
pick_depth  117.04
2100
<class 'str'> type to right of  AA/15-12-097-08W4/0
index_of_neigh_UWI=  2104
pick_depth  68
2101
<class 'str'> type to right of  AA/10-06-098-07W4/0
index_of_neigh_UWI=  2133
pick_depth  75.5
2102
<class 'str'> type to right of  AA/10-06-098-07W4/0
index_of_neigh_UWI=  2133
pick_depth  75.5
2103
<class 'str'> type to right of  AA/16-20-096-08W4/0
index_of_neigh_UWI=  2054
pick_depth  36.5
2104
<c

2177
<class 'str'> type to right of  AA/14-35-100-13W4/0
index_of_neigh_UWI=  2178
pick_depth  572.11
2178
<class 'str'> type to right of  AA/15-14-101-13W4/0
index_of_neigh_UWI=  2185
pick_depth  582
2179
<class 'str'> type to right of  AA/02-29-099-14W4/0
index_of_neigh_UWI=  2170
pick_depth  560
2180
<class 'str'> type to right of  AA/11-08-101-14W4/0
index_of_neigh_UWI=  2189
pick_depth  505
2181
<class 'str'> type to right of  AA/01-05-101-13W4/0
index_of_neigh_UWI=  2184
pick_depth  546.5
2182
<class 'str'> type to right of  AA/06-31-101-12W4/0
index_of_neigh_UWI=  2183
pick_depth  575.5
2183
<class 'str'> type to right of  AA/07-08-101-12W4/0
index_of_neigh_UWI=  2182
pick_depth  660
2184
<class 'str'> type to right of  AA/14-35-100-13W4/0
index_of_neigh_UWI=  2178
pick_depth  572.11
2185
<class 'str'> type to right of  AA/14-35-100-13W4/0
index_of_neigh_UWI=  2178
pick_depth  572.11
2186
<class 'str'> type to right of  AA/11-30-101-13W4/0
index_of_neigh_UWI=  2188
pick_depth  4

In [295]:
temp5 = temp4[['SitID','McMurray_Base_HorID','McMurray_Top_HorID','McMurray_Base_DEPTH','McMurray_Top_DEPTH','McMurray_Base_Qual','McMurray_Top_Qual','lat','lng','UWI','Neighbors_Obj','NN1_McMurray_Top_DEPTH','NN1_McMurray_Base_DEPTH']]

In [297]:
temp5.head()

,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,UWI,Neighbors_Obj,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH
0,102496,14000,13000,561,475,1,3,54.785907,-110.129320,00/12-08-067-01W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 2, 'distance': 0.08749430324884858, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 3, 'distance': 0.10451669032744222, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 4, 'distance': 0.1099989647224123, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 5, 'distance': 0.14017282905400244, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.14539497462086076, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.14676425482044453, 'UWI': '00/10-29-067-02W4/0'}]",544.0,630.0
1,102497,14000,13000,604.5,515,1,3,54.782284,-110.269446,00/07-08-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04734000608365529, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.07332213830078355, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.07725585850924727, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 4, 'distance': 0.07846755926496476, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 5, 'distance': 0.08267376663125782, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 6, 'distance': 0.09089701215112062, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 7, 'distance': 0.11681159403073145, 'UWI': '00/06-13-068-03W4/0'}]",529.0,613.0
2,102498,14000,13000,564,480,1,3,54.785892,-110.186851,00/09-11-067-02W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/12-08-067-01W4/0'}, {'neighbor': 2, 'distance': 0.06649378619088428, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.08267376663125782, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.08728400165551212, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 5, 'distance': 0.09343691917544036, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 6, 'distance': 0.10808166919974113, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 7, 'distance': 0.12013967631468725, 'UWI': '00/10-08-068-02W4/0'}]",529.0,613.0
3,102500,14000,13000,636.5,549,1,3,54.829624,-110.269422,00/10-29-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04355700607250212, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 2, 'distance': 0.04589036401030715, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.04734000608365529, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.052016152587450945, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 5, 'distance': 0.07898840474399109, 'UWI': '00/06-13-068-03W4/0'}, {'neighbor': 6, 'distance': 0.08169505420160347, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 7, 'distance': 0.08752301126560716, 'UWI': '00/11-29-068-02W4/0'}]",529.0,613.0
4,102501,14000,13000,613,529,1,2,54.840471,-110.224832,00/06-34-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04589036401030715, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.050134469469623355, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 3, 'distance': 0.055282561346227096, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 4, 'distance': 0.06649378619088428, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 5, 'distance': 0.07332213830078355, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.07715524006184091, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.09180352126144718, 'UWI': '00/11-29-068-02W4/0'}]",514.0,603.0


In [298]:
temp5['NN1_thickness'] = temp5['NN1_McMurray_Base_DEPTH'] - temp5['NN1_McMurray_Top_DEPTH']

In [299]:
temp5.head()

,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,UWI,Neighbors_Obj,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness
0,102496,14000,13000,561,475,1,3,54.785907,-110.129320,00/12-08-067-01W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 2, 'distance': 0.08749430324884858, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 3, 'distance': 0.10451669032744222, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 4, 'distance': 0.1099989647224123, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 5, 'distance': 0.14017282905400244, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.14539497462086076, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.14676425482044453, 'UWI': '00/10-29-067-02W4/0'}]",544.0,630.0,86.0
1,102497,14000,13000,604.5,515,1,3,54.782284,-110.269446,00/07-08-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04734000608365529, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.07332213830078355, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.07725585850924727, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 4, 'distance': 0.07846755926496476, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 5, 'distance': 0.08267376663125782, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 6, 'distance': 0.09089701215112062, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 7, 'distance': 0.11681159403073145, 'UWI': '00/06-13-068-03W4/0'}]",529.0,613.0,84.0
2,102498,14000,13000,564,480,1,3,54.785892,-110.186851,00/09-11-067-02W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/12-08-067-01W4/0'}, {'neighbor': 2, 'distance': 0.06649378619088428, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.08267376663125782, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.08728400165551212, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 5, 'distance': 0.09343691917544036, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 6, 'distance': 0.10808166919974113, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 7, 'distance': 0.12013967631468725, 'UWI': '00/10-08-068-02W4/0'}]",529.0,613.0,84.0
3,102500,14000,13000,636.5,549,1,3,54.829624,-110.269422,00/10-29-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04355700607250212, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 2, 'distance': 0.04589036401030715, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.04734000608365529, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.052016152587450945, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 5, 'distance': 0.07898840474399109, 'UWI': '00/06-13-068-03W4/0'}, {'neighbor': 6, 'distance': 0.08169505420160347, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 7, 'distance': 0.08752301126560716, 'UWI': '00/11-29-068-02W4/0'}]",529.0,613.0,84.0
4,102501,14000,13000,613,529,1,2,54.840471,-110.224832,00/06-34-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04589036401030715, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.050134469469623355, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 3, 'distance': 0.055282561346227096, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 4, 'distance': 0.06649378619088428, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 5, 'distance': 0.07332213830078355, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.07715524006184091, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.09180352126144718, 'UWI': '00/11-29-068-02W4/0'}]",514.0,603.0,89.0


In [308]:
def convertStringToFloat(string):
    try:
        string = float(string)
    except:
        string = 0
    return string

temp5['McMurray_Base_DEPTH'] = temp5['McMurray_Base_DEPTH'].apply(convertStringToFloat)

In [309]:


temp5['MM_Top_Depth_predBy_NN1thick'] =  temp5['McMurray_Base_DEPTH'] - temp5['NN1_thickness'] 

In [310]:
temp5.head()

,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,UWI,Neighbors_Obj,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick
0,102496,14000,13000,561.0,475,1,3,54.785907,-110.129320,00/12-08-067-01W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 2, 'distance': 0.08749430324884858, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 3, 'distance': 0.10451669032744222, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 4, 'distance': 0.1099989647224123, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 5, 'distance': 0.14017282905400244, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.14539497462086076, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.14676425482044453, 'UWI': '00/10-29-067-02W4/0'}]",544.0,630.0,86.0,475.0
1,102497,14000,13000,604.5,515,1,3,54.782284,-110.269446,00/07-08-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04734000608365529, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.07332213830078355, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.07725585850924727, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 4, 'distance': 0.07846755926496476, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 5, 'distance': 0.08267376663125782, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 6, 'distance': 0.09089701215112062, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 7, 'distance': 0.11681159403073145, 'UWI': '00/06-13-068-03W4/0'}]",529.0,613.0,84.0,520.5
2,102498,14000,13000,564.0,480,1,3,54.785892,-110.186851,00/09-11-067-02W4/0,"[{'neighbor': 1, 'distance': 0.05753100195547899, 'UWI': '00/12-08-067-01W4/0'}, {'neighbor': 2, 'distance': 0.06649378619088428, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.08267376663125782, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.08728400165551212, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 5, 'distance': 0.09343691917544036, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 6, 'distance': 0.10808166919974113, 'UWI': '00/11-08-068-01W4/0'}, {'neighbor': 7, 'distance': 0.12013967631468725, 'UWI': '00/10-08-068-02W4/0'}]",529.0,613.0,84.0,480.0
3,102500,14000,13000,636.5,549,1,3,54.829624,-110.269422,00/10-29-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04355700607250212, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 2, 'distance': 0.04589036401030715, 'UWI': '00/06-34-067-02W4/0'}, {'neighbor': 3, 'distance': 0.04734000608365529, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 4, 'distance': 0.052016152587450945, 'UWI': '00/07-36-067-03W4/0'}, {'neighbor': 5, 'distance': 0.07898840474399109, 'UWI': '00/06-13-068-03W4/0'}, {'neighbor': 6, 'distance': 0.08169505420160347, 'UWI': '00/10-14-067-03W4/0'}, {'neighbor': 7, 'distance': 0.08752301126560716, 'UWI': '00/11-29-068-02W4/0'}]",529.0,613.0,84.0,552.5
4,102501,14000,13000,613.0,529,1,2,54.840471,-110.224832,00/06-34-067-02W4/0,"[{'neighbor': 1, 'distance': 0.04589036401030715, 'UWI': '00/10-29-067-02W4/0'}, {'neighbor': 2, 'distance': 0.050134469469623355, 'UWI': '00/09-11-068-02W4/0'}, {'neighbor': 3, 'distance': 0.055282561346227096, 'UWI': '00/10-08-068-02W4/0'}, {'neighbor': 4, 'distance': 0.06649378619088428, 'UWI': '00/09-11-067-02W4/0'}, {'neighbor': 5, 'distance': 0.07332213830078355, 'UWI': '00/07-08-067-02W4/0'}, {'neighbor': 6, 'distance': 0.07715524006184091, 'UWI': '00/06-26-068-02W4/0'}, {'neighbor': 7, 'distance': 0.09180352126144718, 'UWI': '00/11-29-068-02W4/0'}]",514.0,603.0,89.0,524.0


In [311]:
wells_df_new_cleaned_plus_nn = temp5.copy()
#### dumping tree to pickle file
pickle.dump( wells_df_new_cleaned_plus_nn, open( "wells_df_new_cleaned_plus_nn.p", "wb" ) )

In [262]:
test = df_new_cleaned_plus_nn.iloc[62,4]

In [263]:
test

'544'

### Future work

In [12]:
#### 1. Find average pick depth of N neighbor wells within a certain maximum distance
#### takes in array or pandas df? returns dataframe?
def avgPickDepthForKneighborsWcertainDist(array,k_neighbors,max_distance):
    
    
    return uwi_array, distance_array
    

In [13]:
#### 1. Find average pick depth of N neighbor wells within a certain maximum distance
#### takes in array or pandas df? returns dataframe?
def avgPickDepthForKneighborsWcertainDist(array,k_neighbors,max_distance):
    return uwi_nearest, uwi_array, nearest_TopM_depth, avg_TopM_depth, nearest_BaseM_depth, avg_BaseM_depth
    

In [9]:
#### 2. Return an array of pick depths at N neighbor wells within a certain maximum distance.
#### takes in array or pandas df? returns dataframe?
def arrayOfPickDepthForKneighborsWcertainDist(array,k_neighbors,max_distance):
    return array_of_depths
    

In [10]:
#### 3. Return the average depth above or below another known pickin N nearest neighbor wells
#### takes in array or pandas df? returns dataframe?
def avgDepthAbvOrBelowKneighborsWcertainDist(array,k_neighbors,max_distance,aboveOrbelow,pickA,pickB):
    return avg_of_depthsAbvBelow

In [11]:
#### 4. Return the average depth above or below another known pickin N nearest neighbor wells
#### takes in array or pandas df? returns dataframe?
def arrayDepthAbvOrBelowKneighborsWcertainDist(array,k_neighbors,max_distance,aboveOrbelow,pickA,pickB):
    return array_of_depthsAbvBelow